# Airline Ticket Assistant

A Jupyter Notebook for an AI-powered assistant that helps users explore airline ticket options, pricing, and itineraries. This notebook is organized for readability and ease of reuse when shared on GitHub.

- Purpose: prototype and demonstrate an airline ticket assistant workflow
- Scope: data input, model prompts/calls, and result formatting
- Audience: learners and contributors reviewing or extending this project


## Table of Contents

1. Setup
2. Configuration
3. How to Use
4. Results and Notes
5. Notebook Settings
6. Health Checks
7. Imports
8. Constants
9. Initialization
10. Helpers/Utilities
11. Core Assistant Logic
12. Results Rendering


## Setup

Follow these steps to recreate the environment locally.

1. Create and activate a virtual environment:
```bash
python -m venv .venv
.venv\Scripts\activate
```
2. Install dependencies:
```bash
pip install -r requirements.txt
```
3. (Optional) Install Jupyter extensions for nicer rendering:
```bash
pip install jupyterlab
```
4. Start Jupyter:
```bash
jupyter notebook
```

If you don't have a `requirements.txt` yet, you can export a minimal one later:
```bash
pip freeze > requirements.txt
```


## Configuration

Store secrets and configuration in environment variables and do not commit them.

- Create a `.env` file in the project root (not committed):
```bash
# example
OPENAI_API_KEY=sk-...
AZURE_OPENAI_ENDPOINT=...
AZURE_OPENAI_KEY=...
```
- Load variables in Python (example):
```python
import os
from dotenv import load_dotenv
load_dotenv()
openai_key = os.getenv("OPENAI_API_KEY")
```
- Add these to `.gitignore`:
```
.env
```


## How to Use

Recommended run order and key cells:

1. Configuration: ensure environment variables are loaded
2. Data/Input: set origin, destination, dates, passengers
3. Model/Agent: run the core assistant logic
4. Results: view parsed itineraries and pricing

Tips:
- Keep outputs lightweight; clear outputs before committing
- If using paid APIs, mock small examples when sharing publicly
- Pin package versions in `requirements.txt` for reproducibility


## Results and Notes

Use this section to summarize findings and known limitations.

- Summary: key insights from the latest run
- Limitations: rate limits, data coverage, model behavior
- Next steps: improvements or TODOs for contributors


In [ ]:
# Notebook settings: display, warnings, randomness
import warnings
import random
import numpy as np

warnings.filterwarnings("ignore")
np.set_printoptions(suppress=True, precision=4)
random.seed(42)
np.random.seed(42)

# Optional: widen pandas display if used
try:
    import pandas as pd
    pd.set_option("display.max_columns", 120)
    pd.set_option("display.width", 140)
except Exception:
    pass


In [ ]:
# Health checks: Python, platform, and common deps
import sys, platform, pkgutil
print({
    "python": sys.version.split(" ")[0],
    "platform": platform.platform(),
})

key_packages = ["ipykernel", "ipython", "jupyter", "python-dotenv", "openai", "pandas", "numpy", "requests"]
installed = {name for _, name, _ in pkgutil.iter_modules()}
missing = [p for p in key_packages if p not in installed]
print({"missing": missing})
if missing:
    print("Tip: pip install -r requirements.txt")


## Imports


In [ ]:
# Imports
import os
import json
import string
import random
from dotenv import load_dotenv
import requests 
import gradio as gr
from openai import OpenAI
from bs4 import BeautifulSoup
import ollama
from IPython.display import Markdown , display


## Constants


In [ ]:
# Constants
OLLAMA_API = "http://localhost:11434/api/chat"
HEADERS = {"Content-Type": "application/json"}
MODEL = "llama3.2"


## Initialization


In [ ]:
# Initialization
load_dotenv(override=True)

api_key = os.getenv('OPEN_ROUTER_KEY')
if api_key and api_key.startswith('sk-or-v1') and len(api_key)>10:
   print("API key looks good so far")
else:
   print("There might be a problem with your API key? Please visit the troubleshooting notebook!")
  
openai = OpenAI(
    api_key=api_key,
    base_url="https://openrouter.ai/api/v1"
)    
    
MODEL_Gemini2FlashLite = 'google/gemini-2.0-flash-lite-preview-02-05:free'
MODEL_Gemini2Flash='google/gemini-2.0-flash-exp:free'
MODEL_Gemini2FlashThink = 'google/gemini-2.0-flash-thinking-exp:free'
MODEL_Gemini2Pro ='google/gemini-2.0-pro-exp-02-05:free'
MODEL_Meta_Llama33 ='meta-llama/llama-3.3-70b-instruct:free'
MODEL_Deepseek_V3='deepseek/deepseek-chat:free'
MODEL_Deepseek_R1='deepseek/deepseek-r1-distill-llama-70b:free'
MODEL_Qwen_vlplus='qwen/qwen-vl-plus:free'
MODEL_OpenAi_o3mini = 'openai/o3-mini'
MODEL_OpenAi_4o = 'openai/gpt-4o-2024-11-20'
MODEL_Claude_Haiku = 'anthropic/claude-3.5-haiku-20241022'
MODEL_GPT_FREE= 'openai/gpt-oss-120b:free'

Default_Model = MODEL_Gemini2Flash 
# As an alternative, if you'd like to use Ollama instead of OpenAI
# Check that Ollama is running for you locally (see week1/day2 exercise) then uncomment these next 2 lines

# MODEL = 'llama3.2'
# openai = OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')


## Helpers / Utilities

Brief helper functions go here (formatting, validation, small utilities). Keep them pure and testable if possible.


## Core Assistant Logic

Main chat/agent flow, including tool-calling and message handling. Keep I/O separate from logic for easier testing.


## Results Rendering

Render the final answers, tables, and UI pieces. Keep rendering separate from computation for clarity.


In [ ]:
### Optional: Ensure Ollama model is available

# If using Ollama locally, pull the model first (run this in a terminal, not in the notebook):


!ollama pull llama3.2



In [ ]:
system_message = "You are FlightAI, a friendly and professional airline ticket assistant. "
system_message += "Always start by greeting the user warmly before anything else. "
system_message += "Do not ask about tickets or flight details until the user shows interest or asks. "
system_message += "Respond naturally to greetings and casual conversation. "
system_message += "When users ask about flights, use the available tools to help with booking, pricing, availability, and cancellations. "
system_message += "Only use tools when the user specifically asks about flight-related tasks. "
system_message += "Be conversational and helpful, but don't assume what the user wants unless they clearly state it."


In [ ]:

# =============================
# 🛫 Flight Availability Checker
# =============================

# Mock flight database (city → total seats available)
flight_availability = {
    "london": 25,
    "paris": 30,
    "new york": 15,
    "tokyo": 10,
    "dubai": 40,
    "delhi": 50,
    "sydney": 12,
    "toronto": 20,
    "singapore": 18,
    "los angeles": 22
}

def check_availability(destination_city):
    """
    Check if seats are available for the given destination city.
    Returns the number of seats available or 'Unknown' if the city is not in the system.
    """
    print(f"Tool check_availability called for {destination_city}")
    city = destination_city.lower()
    
    if city in flight_availability:
        seats = flight_availability[city]
        if seats > 0:
            return f"Yes, {seats} seats are available for {destination_city.title()}."
        else:
            return f"Sorry, no seats are currently available for {destination_city.title()}."
    else:
        return "Unknown"


In [ ]:
# Example: check availability
# check_availability("delhi")


Instead of storing exact values, generate prices within a realistic range per city:

Prices feel more dynamic, not the same every time.

Still controlled (not random nonsense, but realistic ranges).

Closer to how real airline prices fluctuate.

In [ ]:
# ======================
# 💲 Flight Ticket Price
# ======================

ticket_price_ranges = {
    "valletta": (700, 850),
    "turin": (800, 950),
    "sacramento": (1300, 1500),
    "montreal": (450, 550),
    "london": (600, 700),
    "paris": (650, 750),
    "new york": (1100, 1300),
    "tokyo": (1400, 1600),
    "sydney": (1700, 1900),
    "dubai": (850, 950),
    "delhi": (500, 600),
    "toronto": (750, 850),
    "singapore": (1000, 1200),
    "los angeles": (1200, 1400)
}

def get_ticket_price(destination_city):
    print(f"Tool get_ticket_price called for {destination_city}")
    city = destination_city.lower()
    if city in ticket_price_ranges:
        low, high = ticket_price_ranges[city]
        price = random.randint(low, high)  # random price in range
        return f"{price} Dollars"
    else:
        return "Unknown"


In [ ]:
# Example: get ticket price
# get_ticket_price("delhi")

In [ ]:
# ========================
# 🎟️ Flight Booking System
# ========================

# Mock booking database
bookings = {
    # Example: "FA-7K8XQ2": {"destination_city": "London", "seats": 1}
}

def book_flight(destination_city, airline_code="FA"):  
    # Check availability first
    city = destination_city.lower()
    if city in flight_availability and flight_availability[city] > 0:
        # Decrease available seats
        flight_availability[city] -= 1
        
        # Airline code default: FlightAI (FA)
        # Allowed characters (excluding confusing ones like O, I, 0, 1)
        letters = "ABCDEFGHJKLMNPQRSTUVWXYZ"
        digits = "23456789"
        
        # Generate booking reference (6 characters: mix of letters and digits)
        booking_code = "".join(random.choice(letters + digits) for _ in range(6))
        
        # Final format: AirlineCode-BookingRef (e.g., FA-7K8XQ2)
        final_code = f"{airline_code}-{booking_code}"
        
        # Store booking in database
        bookings[final_code] = {
            "destination_city": destination_city.title(),
            "seats": 1
        }
        
        print(f"Booking code {final_code} generated for flight to {destination_city}.")
        return f"Flight booked successfully! Your booking code is {final_code} for {destination_city.title()}."
    else:
        return f"Sorry, no seats available for {destination_city.title()}."

In [ ]:
book_flight("delhi")

In [ ]:
# ==========================
# 🎟️ Cancel Flight Function
# ==========================
def cancel_flight(destination_city, booking_code=None):
    """
    Cancel a booked flight for a given destination city.
    If booking_code is provided, it cancels that specific booking.
    Returns a confirmation message.
    """
    city = destination_city.lower()
    
    if booking_code:
        # Cancel using booking code
        if booking_code in bookings:
            if bookings[booking_code]["destination_city"].lower() == city:
                # Return seat to availability
                if city in flight_availability:
                    flight_availability[city] += 1
                del bookings[booking_code]
                return f"Booking {booking_code} for {destination_city.title()} has been successfully cancelled."
            else:
                return f"Booking code {booking_code} does not match the destination {destination_city.title()}."
        else:
            return f"No booking found with code {booking_code}."
    
    else:
        # Cancel any booking for this city (first match)
        for code, info in list(bookings.items()):
            if info["destination_city"].lower() == city:
                # Return seat to availability
                if city in flight_availability:
                    flight_availability[city] += 1
                del bookings[code]
                return f"Booking {code} for {destination_city.title()} has been successfully cancelled."
        return f"No bookings found for {destination_city.title()}."


In [ ]:
# ticket price function
price_function = {
    "name": "get_ticket_price",
    "description": (
        "Retrieve the price of a return ticket to a specific destination city. "
        "Call this when the customer asks about fares. "
        "Example: 'How much is a ticket to Paris?' or 'What's the price for a return ticket to New York?'"
    ),
    "parameters": {
        "type": "object",
        "properties": {
            "destination_city": {
                "type": "string",
                "description": "The city that the customer wants to travel to.",
            },
        },
        "required": ["destination_city"],
        "additionalProperties": False
    }
}



In [ ]:
#Availability Function

availability_function = {
    "name": "check_availability",
    "description": (
        "Check if seats are available for a specific destination city on a given date. "
        "Use this when the customer asks about ticket availability. "
        "Example: 'Are there tickets for Mumbai tomorrow?' or 'Is a seat available to Rome on 2025-10-05?'"
    ),
    "parameters": {
        "type": "object",
        "properties": {
            "destination_city": {
                "type": "string",
                "description": "The city for which ticket availability needs to be checked.",
            },
            "date": {
                "type": "string",
                "description": "The date of travel in YYYY-MM-DD format.",
            },
        },
        "required": ["destination_city", "date"],
        "additionalProperties": False
    }
}


In [ ]:
# Booking Function

book_function = {
    "name": "book_ticket",
    "description": (
        "Book a return ticket for the customer to the requested destination city on a specific date. "
        "Use this when the customer explicitly wants to book. "
        "Example: 'Book me a ticket to London for tomorrow' or 'I need a return ticket to Delhi for 2025-10-05.'"
    ),
    "parameters": {
        "type": "object",
        "properties": {
            "destination_city": {
                "type": "string",
                "description": "The city for which the customer wants to book a ticket.",
            },
            "date": {
                "type": "string",
                "description": "The date of travel in YYYY-MM-DD format.",
            },
        },
        "required": ["destination_city", "date"],
        "additionalProperties": False
    }
}


In [ ]:

cancel_function = {
    "name": "cancel_ticket",
    "description": (
        "Cancel a previously booked ticket for the specified destination city. "
        "Call this when the customer requests cancellation. "
        "Example: 'Cancel my ticket to Berlin' or 'I don’t need the Paris ticket anymore.'"
    ),
    "parameters": {
        "type": "object",
        "properties": {
            "destination_city": {
                "type": "string",
                "description": "The destination city of the ticket that needs to be canceled.",
            },
        },
        "required": ["destination_city"],
        "additionalProperties": False
    }
}


In [ ]:
# ==========================
# ✈️ FlightAI Tools List
# ==========================

tools = [
    {"type": "function", "function": price_function},
    {"type": "function", "function": book_function},
    {"type": "function", "function": cancel_function},
    {"type": "function", "function": availability_function}
]


In [ ]:


def handle_tool_call(message):
    """
    Handles tool calls from the LLM and executes the corresponding FlightAI functions.
    Returns a list of responses and optional additional data (like images if needed).
    """
    responses = []
    
    for tool_call in message.tool_calls:
        # Parse arguments from the LLM tool call
        arguments = json.loads(tool_call.function.arguments)
        function_name = tool_call.function.name
        
        # Match the function name to actual implementation
        if function_name == 'get_ticket_price':
            destination_city = arguments.get('destination_city')
            outdata = get_ticket_price(destination_city)
            input_name = "destination_city"
            output_name = "price"
        
        elif function_name == 'book_ticket':
            # Map to actual function book_flight
            destination_city = arguments.get('destination_city')
            outdata = book_flight(destination_city)
            input_name = "destination_city"
            output_name = "booking_code"
        
        elif function_name == 'cancel_ticket':
            # Map to actual function cancel_flight
            destination_city = arguments.get('destination_city')
            outdata = cancel_flight(destination_city)
            input_name = "destination_city"
            output_name = "cancellation_status"
        
        elif function_name == 'check_availability':
            destination_city = arguments.get('destination_city')
            outdata = check_availability(destination_city)
            input_name = "destination_city"
            output_name = "availability"
        
        else:
            # fallback if unknown tool
            outdata = f"Function {function_name} not recognized."
            input_name = "input"
            output_name = "output"

        # Append response for this tool call
        responses.append({
            "role": "tool",
            "content": json.dumps({input_name: arguments.get('destination_city', 'unknown'), output_name: outdata}),
            "tool_call_id": tool_call.id
        })

    return responses


In [ ]:
def chat(history):
    """
    Main chat function for FlightAI assistant.
    Takes Gradio history format and handles tool calls automatically.
    """
    if not history:
        return history
    
    # Convert Gradio history format to OpenAI format
    messages = [{"role": "system", "content": system_message}]
    
    for msg in history:
        if msg["role"] == "user":
            messages.append({"role": "user", "content": msg["content"]})
        elif msg["role"] == "assistant":
            messages.append({"role": "assistant", "content": msg["content"]})

    # Call the LLM with tools
    response = openai.chat.completions.create(
        model=MODEL,
        messages=messages,
        tools=tools  # your FlightAI tool schemas
    )

    # If LLM decides to call a tool
    if response.choices[0].finish_reason == "tool_calls":
        tool_message = response.choices[0].message

        # Handle the tool call using your FlightAI functions
        tool_responses = handle_tool_call(tool_message)

        # Append tool call and its response to the message history
        messages.append({
            "role": "assistant",
            "content": tool_message.content or "",
            "tool_calls": [{"id": tc.id, "type": tc.type, "function": {"name": tc.function.name, "arguments": tc.function.arguments}} for tc in tool_message.tool_calls]
        })
        
        for r in tool_responses:
            # each r is a dict with role "tool" and content json
            messages.append({"role": "tool", "content": r["content"], "tool_call_id": r["tool_call_id"]})

        # Call LLM again with updated history to get final chat response
        response = openai.chat.completions.create(
            model=MODEL,
            messages=messages
        )

    # Add assistant response to history and return
    assistant_message = response.choices[0].message.content
    history.append({"role": "assistant", "content": assistant_message})
    
    return history


In [ ]:
import gradio as gr
import json

# ===========================
# FlightAI Professional UI
# ===========================

# Assume these are already defined:
# - system_message
# - tools
# - chat(message, history)
# - handle_tool_call(message)
# - get_ticket_price, book_ticket, cancel_flight, check_availability

with gr.Blocks(
    css="""
    .red-button {
        background-color: darkred !important;
        border-color: red !important;
        color: white !important;
    }
    .green-button {
        background-color: darkgreen !important;
        border-color: green !important;
        color: white !important;
    }
    """
) as ui:

    gr.Markdown("<h1 style='text-align:center'>✈️ FlightAI Ticket Assistant</h1>")

    # Chatbot interface
    chatbot = gr.Chatbot(height=500, type="messages")

    # Text input for chat
    entry = gr.Textbox(
        label="Chat with FlightAI:",
        placeholder="Ask about ticket prices, availability, or book/cancel a flight..."
    )

    # Buttons for clear chat and book ticket
    with gr.Row():
        clear = gr.Button(value="Clear Chat", elem_classes="red-button")
        book = gr.Button(value="Book Ticket", elem_classes="green-button")

    # =====================
    # Helper functions
    # =====================

    # Add user message to chat and get AI response
    def respond(message, history):
        history = history or []
        # Add user message
        history.append({"role": "user", "content": message})
        # Get AI response
        updated_history = chat(history)
        return "", updated_history

    # Clear chat
    def clear_chat():
        return []

    # Book ticket button adds a pre-filled booking message
    def add_booking_intent(history):
        history = history or []
        # Add user message for booking intent
        history.append({"role": "user", "content": "I want to book a ticket"})
        # Get AI response
        updated_history = chat(history)
        return updated_history

    # =====================
    # Event bindings
    # =====================

    # User submits a message
    entry.submit(respond, inputs=[entry, chatbot], outputs=[entry, chatbot])

    # Clear chat
    clear.click(lambda: clear_chat(), inputs=None, outputs=chatbot, queue=False)

    # Book ticket button
    book.click(add_booking_intent, inputs=chatbot, outputs=chatbot)

# Launch the UI in browser
ui.launch(inbrowser=True)
